<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install naslib

In [ ]:
!git clone https://github.com/automl/NASLib.git
!cd NASLib

In [ ]:
!pip install -e .

In [ ]:
from naslib.optimizers import RandomNAS
from naslib.search_spaces import DartsSearchSpace

# Instantiate the DARTS search space
search_space = DartsSearchSpace()

# Instantiate the random search optimizer with a specified budget
optimizer = RandomNAS(search_space, budget=50)

# Perform the search
optimizer.search()

# Retrieve and print the best architecture found
best_architecture = optimizer.get_best_architecture()
print("Best Architecture:", best_architecture)